In [2]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 4.4 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/Users/sarthak/Documents/Playground/Zoning Search/zonevenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [55]:
import PyPDF2
# Open the PDF file in read binary mode
pdf_file = open('zoning_guide.pdf', 'rb')

# Create a PDF reader object
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Get the total number of pages in the PDF file
num_pages = len(pdf_reader.pages)

# Loop through each page and extract the text
text = ''
for i in range(num_pages):
    page = pdf_reader.pages[i]
    text += page.extract_text()

# Define the size of each chunk (in characters)
chunk_size = 4000

# Break the text into chunks
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

In [56]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('msmarco-distilbert-base-v4')

In [57]:
vectors = model.encode(chunks)

In [16]:
pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.3 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/Users/sarthak/Documents/Playground/Zoning Search/zonevenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [58]:
import faiss
import numpy as np
# Create a FAISS index

index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

('n may \nbe applied  to a use in a residential  neighborhood that contains  a mixture  of single  family  and \nmultifamily  uses or in an area for which limited  density  multifamily  use is  desired.  An MF -1 \ndistrict may be used as a transition  between  a single  family  and higher  intensity  uses.  \n \n \nSite Development  Standards  \n \nLot  Massing  \nMinimum  Lot Size 8,000 sq ft Maximum  Height  40 ft \nMinimum  Lot Width 50 ft Minimum  Setbacks   \nMaximum  Building  Coverage  45% Front  yard 25 ft \nMaximum  Impervious  Cover 55% Street  side yard 15 ft \nMaximum  Floor  Area  Ratio  n/a Interior  side yard 5 ft \nMaximum  Units  Per Acre  17 Rear  yard 10 ft \n \nPermitted  and Conditional  Uses  \n†  For detail on marked  uses in  the Multi -Family Residence—Limited  Density  district , see the \nAustin  City Code  25-2-560. \n \nResidential \nBed and Breakfast Residential  (Group 1) * Retirement  Housing  (Large  Site)  (c) * \nBed and Breakfast  Residential  (Grou

In [59]:
def search(query, index, vectors, k=5):
    query_vector = model.encode([query])[0]
    distances, indices = index.search(np.array([query_vector]), k)
    return [(chunks[i], distances[0][j]) for j, i in enumerate(indices[0])]

# Perform a search for the query "machine learning"
results = search("Single Family Zoning", index, vectors)

# Print the top five results
for i, (result, distance) in enumerate(results):
    print(f"Result {i+1}: {result}\nDistance: {distance}\n")


Result 1: n  single  family  neighborhood characteristics.  
 
 
Site Development  Standards  
 
Lot                                                                 Massing  
Minimum  Lot Size                                 3,600 sq ft      Maximum  Height                                       35 ft 
Minimum  Lot Width                                   40 ft          Minimum  Setbacks  
Maximum  Units  Per Lot                               1                Front  yard                                                15 ft 
Maximum  Building Cov erage                   55%              Street  side yard                                         10 ft 
Maximum  Impervious  Cover                  65%              Interior  side yard                                      — 
Rear  yard      — 
 
 
Permitted  and Conditional  Uses  
 
Residential 
Small  Lot Single -Family Residential  *  Short -Term Rental  
 
Civic  
Club  or Lodge  (c) * 
College  and University  Facilities  (c) * 
Communicat

In [ ]:
TestKEY = ""

In [22]:
import os
import openai
openai.api_key = ""
openai.Model.list()

<OpenAIObject list at 0x107b7f770> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl

In [70]:
def Querythis(InputQuestions):
    
    results = search(InputQuestions, index, vectors)
    introduction1 = 'Use the below articles on the Austin Zoning rules to answer the following question:'
    introduction2 = 'If the answer cannot be found in the articles, write "I could not find an answer."'
    
    message = introduction1 + InputQuestions + introduction2 + str(results[0]) + str(results[1]) + str(results[2]) 
    messages = [
        {"role": "system", "content": "You answer questions about the zoning code for the city of Austin."},
        {"role": "user", "content": message},
    ]
    # print(message)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [71]:
print(Querythis("How do I request a zoning change for the city of Austin?"))

To request a zoning change for the city of Austin, one must submit an application to the Development Services Department. The application must include a completed zoning change application form, a legal description of the property, a site plan, and a fee. The application will then be reviewed by city staff and forwarded to the Planning Commission for a recommendation and then to the City Council for a final decision. (Source: https://www.austintexas.gov/faq/how-do-i-request-zoning-change)


In [69]:
results = search("How do I request a zoning change for the city of Austin?", index, vectors)
for i, (result, distance) in enumerate(results):
    print(f"Result {i+1}: {result}\nDistance: {distance}\n")

Result 1:  
 
 
City of Austin 
 
 
 
  
 
  
 
  
 
Guide to Zoning    
 
 
 
 
 
 
 
 
 
  
 
  
 
  
 
  
 
City of Austin  
 
 
Planning & Zoning Department  
505 Barton Springs  Road 
One Texas  Center 
Austin,  TX 78704  
 
 
September 2016  
 
 
 
  
 
  
 
  
 
 
 
 
  
 
 
 
DISCLAIMER:  This Guide  is for informational  purposes onl y. It does not  constitute  legal 
regulations. Consult  the Land  Development  Code  for specific  regulations:  
                http://www.austintexas.gov/department/austin -city-code- land-development -code . City of  Austin  1  September  2016  Zoning  Guide  
 
 
 
  
 
Contents  
 
 
I. Introduction to Zoning   2 
II. Zoning Principles   5 
III. Zoning Process   6 
IV.  Challenging a Zoning Request   8 
V.  Base Zoning Districts   11 
Residential Districts   13 
Office Districts   36 
Commercial Districts   42 
Industrial Districts   61 
Special  Purpose Districts   69 
VI. Combining Districts   72 
VII.  Description of Allow ed  Uses    78